# Tuning (Hyperparameter Optimization)

Although it doesn't appear in our high-level outline diagram, __model tuning__ is the next critical step in a typical data-to-deployment ML flow.

Tuning can take many forms
* Choosing regularization within a narrow family of models
    * L1 and L2 components ("ElasticNet") applied to a linear model
    * Number of trees or max tree depth in a tree ensemble like random forest or gradient-boosted trees
* Adjusting a a parameter which alters the modeling family significantly
    * Polynomial order in polynomial regression
    * Kernel choice in kernelized SVMs
* Architecture search
    * Layer type and size in a neural network

... and more, depending on the assumptions of the team and tools.

While early tuning approaches used grid search (searching a grid of "points in hyperparam space" to find the best performing model) or random search (within hyperparam space), more recent libraries have expanded accessibility to sophisticated tuning approaches including
* Hyperband (a bandit-based approach)
* Bayesian
* Population based
and more.

This has, in turn, spawned frameworks, like Optuna and HyperOpt to encapsulate these techniques.

## Tuning is easy ... and hard

Computationally, most tuning approaches are embarrasingly parallel operations. This means that they can be fairly easily scaled to many experiments in parallel, taking advantage of large-scale compute to get results quickly.

At the same time, as model and tuning complexity increase, there is a rise in value for tooling that can manage, track, and automate this tuning.

## Dask and Ray

Dask supports a number of approaches to tuning as described here: https://ml.dask.org/hyper-parameter-search.html

The Dask approach is most valuable to users who want to get their hands on the pipeline and programmatically manage model training and hyperparam search.

Ray takes a slightly different angle: as we saw with Ray RLlib, which encapsulates training use cases into a high-level interface, Ray prominently features a similar tuning library: __Ray Tune__ (https://docs.ray.io/en/latest/tune/)

This example -- from the Ray project documentation -- shows the high-level structure/flow:

In [ ]:
from ray import tune

def objective(step, alpha, beta):
    return (0.1 + alpha * step / 100)**(-1) + beta * 0.1

def training_function(config):
    # Hyperparameters
    alpha, beta = config["alpha"], config["beta"]
    for step in range(10):
        # Iterative training function - can be any arbitrary training procedure.
        intermediate_score = objective(step, alpha, beta)
        # Feed the score back back to Tune.
        tune.report(mean_loss=intermediate_score)

analysis = tune.run(
    training_function,
    config={
        "alpha": tune.grid_search([0.001, 0.01, 0.1]),
        "beta": tune.choice([1, 2, 3])
    })

print("Best config: ", analysis.get_best_config(
    metric="mean_loss", mode="min"))

# Get a dataframe for analyzing trial results.
df = analysis.results_df

We'll see a more realistic example in the lab, but the key point here is that Ray Tune exposes a meta-API over the underlying algorithms, so that we can more quickly and simply scale lots of experiments.